In [1]:
import json
import numpy as np
import pandas as pd
from pymongo import MongoClient
import src.filtering as fn
import src.query_fun as mf

In [2]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

### Obtenemos de la colección "companies" las oficinas que cuentan con los apartados:
#### funding_rounds, offices, latitude, longitude, deadpooled_year (que no sea mayor o igual que 0 )

In [3]:
companies = list(db["companies"].find({"funding_rounds":{"$exists": True, "$not": {"$size": 0}},"offices":{"$exists": True, "$not": {"$size": 0}}, "offices.latitude":{"$exists":True, "$ne":None}, "offices.longitude":{"$exists":True, "$ne":None},"deadpooled_year":{"$not":{"$gte":0}}},{"name":1,"offices":1,"category_code":1, "funding_rounds":1, "founded_year":1}))
df = pd.DataFrame(companies)
df.head()

,_id,name,category_code,founded_year,funding_rounds,offices
0,52cdef7c4bab8bd675297d91,Geni,web,2006.0,"[{'id': 6, 'round_code': 'a', 'source_url': ''...","[{'description': 'Headquarters', 'address1': '..."
1,52cdef7c4bab8bd675297d96,Gizmoz,web,2003.0,"[{'id': 9, 'round_code': 'a', 'source_url': 'h...","[{'description': None, 'address1': None, 'addr..."
2,52cdef7c4bab8bd675297d8d,Digg,news,2004.0,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...","[{'description': None, 'address1': '135 Missis..."
3,52cdef7c4bab8bd675297d99,Lala,games_video,NaN,"[{'id': 16, 'round_code': 'b', 'source_url': '...","[{'description': 'Lala Headquarters', 'address..."
4,52cdef7c4bab8bd675297d98,Slacker,music,2006.0,"[{'id': 12, 'round_code': 'b', 'source_url': '...","[{'description': '', 'address1': '16935 W. Ber..."


In [4]:
#Encontramos los diferentes códigos de los países y elegimos Canadá
#query={'offices': { '$exists': 'true', '$not': {'$size': 0} }}
#web_offices = list(db.companies.find(query,{"offices":1,"name":1}))

#countries=set()
#for i,e in enumerate(web_offices):
    #countries.add(e['offices'][0]['country_code'])
#countries #CAN es el de Canadá

#### Sumamos el total de funding_rounds y obtenemos la columna total_funding para saber el total de dinero invertido en USD.

In [5]:
total_funding = fn.sum_funding(df.funding_rounds)
df["total_funding"] = total_funding
df.head()

,_id,name,category_code,founded_year,funding_rounds,offices,total_funding
0,52cdef7c4bab8bd675297d91,Geni,web,2006.0,"[{'id': 6, 'round_code': 'a', 'source_url': ''...","[{'description': 'Headquarters', 'address1': '...",16500000.0
1,52cdef7c4bab8bd675297d96,Gizmoz,web,2003.0,"[{'id': 9, 'round_code': 'a', 'source_url': 'h...","[{'description': None, 'address1': None, 'addr...",18100000.0
2,52cdef7c4bab8bd675297d8d,Digg,news,2004.0,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...","[{'description': None, 'address1': '135 Missis...",45000000.0
3,52cdef7c4bab8bd675297d99,Lala,games_video,NaN,"[{'id': 16, 'round_code': 'b', 'source_url': '...","[{'description': 'Lala Headquarters', 'address...",44150000.0
4,52cdef7c4bab8bd675297d98,Slacker,music,2006.0,"[{'id': 12, 'round_code': 'b', 'source_url': '...","[{'description': '', 'address1': '16935 W. Ber...",73100000.0


#### Utilizamos "explode" para transformar cada elemento de la lista de la fila offices a distintas filas, replicando los valores índice y, por otro lado utilizamos result_type = "expand" para transformar los resultados "list-like" en columnas.
#### Al concatenar todo obtenemos un dataframe formado por más columnas que contienen toda la información que necesitamos.

In [6]:
df = df.explode('offices')
df_offices= df[["offices"]].apply(lambda r: r.offices, result_type="expand", axis=1)
df = pd.concat([df,df_offices], axis=1)
df.head()

,_id,name,category_code,founded_year,funding_rounds,offices,total_funding,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,52cdef7c4bab8bd675297d91,Geni,web,2006.0,"[{'id': 6, 'round_code': 'a', 'source_url': ''...","{'description': 'Headquarters', 'address1': '9...",16500000.0,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064
1,52cdef7c4bab8bd675297d96,Gizmoz,web,2003.0,"[{'id': 9, 'round_code': 'a', 'source_url': 'h...","{'description': None, 'address1': None, 'addre...",18100000.0,None,None,None,None,Menlo Park,CA,USA,37.484130,-122.169472
2,52cdef7c4bab8bd675297d8d,Digg,news,2004.0,"[{'id': 1, 'round_code': 'b', 'source_url': 'h...","{'description': None, 'address1': '135 Mississ...",45000000.0,None,135 Mississippi St,,94107,San Francisco,CA,USA,37.764726,-122.394523
3,52cdef7c4bab8bd675297d99,Lala,games_video,NaN,"[{'id': 16, 'round_code': 'b', 'source_url': '...","{'description': 'Lala Headquarters', 'address1...",44150000.0,Lala Headquarters,209 Hamilton Ave,Suite #200,94301,Palo Alto,CA,USA,37.451151,-122.154369
4,52cdef7c4bab8bd675297d98,Slacker,music,2006.0,"[{'id': 12, 'round_code': 'b', 'source_url': '...","{'description': '', 'address1': '16935 W. Bern...",73100000.0,,16935 W. Bernardo Dr. Suite 101,,92127,San Diego,CA,USA,33.022176,-117.081406


#### Eliminamos las columnas que no nos interesan

In [7]:
df = df.drop(columns = ["funding_rounds", "description", "address2", "state_code", "country_code"])

#### Creamos la columna location como un diccionario de tipo "point" para poder realizar geoqueries con MongoDB.

In [8]:
df["location"] = df[["latitude","longitude"]].apply(lambda x:fn.toGeoJSON(x.latitude,x.longitude), axis=1)
df.head()

,_id,name,category_code,founded_year,offices,total_funding,address1,zip_code,city,latitude,longitude,location
0,52cdef7c4bab8bd675297d91,Geni,web,2006.0,"{'description': 'Headquarters', 'address1': '9...",16500000.0,9229 W. Sunset Blvd.,90069,West Hollywood,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
1,52cdef7c4bab8bd675297d96,Gizmoz,web,2003.0,"{'description': None, 'address1': None, 'addre...",18100000.0,None,None,Menlo Park,37.484130,-122.169472,"{'type': 'Point', 'coordinates': [-122.169472,..."
2,52cdef7c4bab8bd675297d8d,Digg,news,2004.0,"{'description': None, 'address1': '135 Mississ...",45000000.0,135 Mississippi St,94107,San Francisco,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,..."
3,52cdef7c4bab8bd675297d99,Lala,games_video,NaN,"{'description': 'Lala Headquarters', 'address1...",44150000.0,209 Hamilton Ave,94301,Palo Alto,37.451151,-122.154369,"{'type': 'Point', 'coordinates': [-122.154369,..."
4,52cdef7c4bab8bd675297d98,Slacker,music,2006.0,"{'description': '', 'address1': '16935 W. Bern...",73100000.0,16935 W. Bernardo Dr. Suite 101,92127,San Diego,33.022176,-117.081406,"{'type': 'Point', 'coordinates': [-117.081406,..."


#### Vuelvo a eliminar columnas que me doy cuenta de que tampoco son necesarias

In [9]:
df = df.drop(columns=["_id","address1","zip_code"])
df.head()

,name,category_code,founded_year,offices,total_funding,city,latitude,longitude,location
0,Geni,web,2006.0,"{'description': 'Headquarters', 'address1': '9...",16500000.0,West Hollywood,34.090368,-118.393064,"{'type': 'Point', 'coordinates': [-118.393064,..."
1,Gizmoz,web,2003.0,"{'description': None, 'address1': None, 'addre...",18100000.0,Menlo Park,37.484130,-122.169472,"{'type': 'Point', 'coordinates': [-122.169472,..."
2,Digg,news,2004.0,"{'description': None, 'address1': '135 Mississ...",45000000.0,San Francisco,37.764726,-122.394523,"{'type': 'Point', 'coordinates': [-122.394523,..."
3,Lala,games_video,NaN,"{'description': 'Lala Headquarters', 'address1...",44150000.0,Palo Alto,37.451151,-122.154369,"{'type': 'Point', 'coordinates': [-122.154369,..."
4,Slacker,music,2006.0,"{'description': '', 'address1': '16935 W. Bern...",73100000.0,San Diego,33.022176,-117.081406,"{'type': 'Point', 'coordinates': [-117.081406,..."


#### Creamos un json que almacenamos en la carpeta OUTPUT y lo importamos a Mongo DB como una nueva bbdd "Cleaned_offices" con la correspondiente colección llamada  "cleaned_offices".

In [10]:
#df.to_json("cleaned_offices.json", orient="records")

In [21]:
#Incorporamos nueva colección a la bbdd y creamos el índice geoloc 
#!mongoimport --db companies --collection cleaned_offices --drop --jsonArray output/cleaned_offices.json


2020-11-14T19:58:56.823+0100	connected to: mongodb://localhost/
2020-11-14T19:58:56.824+0100	dropping: companies.cleaned_offices
2020-11-14T19:58:56.954+0100	3257 document(s) imported successfully. 0 document(s) failed to import.
